In [1]:
pip install arxiv

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://www.langchain.com/langsmith")
docs= loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever=vectordb.as_retriever()
retriever

c:\Users\Uyama\Downloads\LANGCHAIN-PROJECTS\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002153D572450>)

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about Langsmith. For any questions about LangSmith, you must use this tool")


In [12]:
retriever_tool.name

'langsmith_search'

In [13]:
#Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper)
arxiv.name

'arxiv'

In [14]:
tools=[wiki,arxiv,retriever_tool]

In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Uyama\\Downloads\\LANGCHAIN-PROJECTS\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about Langsmith. For any questions about LangSmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000021501718EA0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorsto

In [16]:
#initialize LLM model
from dotenv import load_dotenv

load_dotenv()
import os

os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [17]:
from langchain import HuggingFaceHub


In [18]:
from langchain_huggingface import HuggingFaceEndpoint
repo_id="mistralai/Mistral-7B-Instruct-v0.3"
llm=HuggingFaceEndpoint(repo_id=repo_id,max_length=150,temperature=0.7,token=os.getenv("HF_TOKEN"))
prompt = llm("Write a 4 line poem on my love for falafel")
print(prompt)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
C:\Users\Uyama\AppData\Local\Temp\ipykernel_20164\136233371.py:4: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  prompt = llm("Write a 4 line poem on my love for falafel")


:

In the heart of the city, a scent so divine,
Falafel, my passion, a culinary wine.
Crunchy, green, with a zest so fine,
A love for falafel, forever thine.


In [19]:
from langchain.agents import initialize_agent, Tool, AgentType
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Choose agent type
    verbose=True,
)

prompt


C:\Users\Uyama\AppData\Local\Temp\ipykernel_20164\1105988591.py:2: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(


':\n\nIn the heart of the city, a scent so divine,\nFalafel, my passion, a culinary wine.\nCrunchy, green, with a zest so fine,\nA love for falafel, forever thine.'

In [20]:
#Agent Executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True,return_intermediate_steps=False)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input', 'page_content'], template="Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_communit

In [21]:
agent_executor.invoke({"input": "Tell me about Langsmith"})
# agent_executor.run("Tell me about Langsmith")



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'

In [124]:
agent_executor.invoke({"input": "Tell me about Machine Learning"})



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'